# Soft Max

softmax는 binary classification을 여러 번 결합한 결과다. 예측 결과가 A, B, C 중의 하나가 되어야 한다면, 동일한 x에 대해 A가 될 확률, B가 될 확률, C가 될 확률을 모두 구해야 한다. x는 3번 사용되지만, W는 A, B, C에 대해서 한 번씩 필요하니까 3번 반복된다. 그래서, W는 예측해야 하는 숫자만큼 필요하게 된다. 10개 중에서 예측한다면 10개의 W가 나와야 한다.

$$
\mathbf{S(Y_i)} = \frac{e^y_i}{\sum\limits_{j=1}^{m}{e^y_j}}
$$

# One-Hot Encoding

ML에서 결과로 보통 두개의 근접한 값들이 거리가 있는 두 개의 값들보다 더 유사하다고 가정하는데, 
예를 들면 결과값이 1인 경우 3보다는 2와 비슷하다고 여겨지는 것을 말합니다.

그런데 ML 예측의 결과로 A, B, C 같이 전혀 관련성이 없는 분류들 중에 하나를 결과로 내놓는 경우는
결과값이 숫자이기 보다는 A, B, C 같은 결과로 도출되길 원할 경우 원-핫 인코딩 기법을 사용합니다.

설명은 장황하지만, 간단히 요약하자면 메모리에서 하나의 비트들을 각 결과로 매칭시켜 A와 유사할 경우
A에 해당하는 bit를 1, 아닐 경우 0 같은 방식으로 처리하는 것을 말합니다

예측결과 A로 예상 = 100(A__)<br>
예측결과 B로 예상 = 010(_B_)<br>
예측결과 C로 예상 = 001(__C)<br>

# Cross-Entropy Cost Function

$$
[S(=y): softmax값,\ L(=\bar{y}): 0\ or\ 1]\\
D(S, L) = -\sum\limits_i L_{i}\log{(S_i)} \\
Cost = \frac{1}{N} \sum_{i=1}^N {D(S(wx_i + b), L_i)}
$$

In [ ]:
import tensorflow as tf

x_data = [[1, 2, 1, 1], [2, 1, 3, 2], [3, 1, 3, 4], [4, 1, 5, 5], [1, 7, 5, 5], [1, 2, 5, 6], [1, 6, 6, 6], [1, 7, 7, 7]]
y_data = [[0, 0, 1], [0, 0, 1], [0, 0, 1], [0, 1, 0], [0, 1, 0], [0, 1, 0], [1, 0, 0], [1, 0, 0]]

X = tf.placeholder("float", [None, 4])
Y = tf.placeholder("float", [None, 3])
nb_classes = 3

W = tf.Variable(tf.random_normal([4, nb_classes]), name='weight')
b = tf.Variable(tf.random_normal([nb_classes]), name='bias')

# tf.nn.softmax computes softmax activations
# softmax = exp(logits) / reduce_sum(exp(logits), dim)
hypothesis = tf.nn.softmax(tf.matmul(X, W) + b)

# Cross entropy cost/loss
cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(hypothesis), axis=1))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

# Launch graph
with tf.Session() as sess:
   sess.run(tf.global_variables_initializer())

   for step in range(2001):
       sess.run(optimizer, feed_dict={X: x_data, Y: y_data})
       if step % 200 == 0:
           print(step, sess.run(cost, feed_dict={X: x_data, Y: y_data}))


In [1]:
# (csv 파일을 이용) 동물들의 특성들을 이용해 동물의 종을 예측
import tensorflow as tf
import numpy as np
tf.set_random_seed(777)  # for reproducibility

# csv 데이터 읽기
xy = np.loadtxt('data-04-zoo.csv', delimiter=',', dtype=np.float32)
x_data = xy[:, 0:-1]
y_data = xy[:, [-1]]

print(x_data.shape, y_data.shape)

nb_classes = 7  # 0 ~ 6

X = tf.placeholder(tf.float32, [None, 16])
Y = tf.placeholder(tf.int32, [None, 1])  # 0 ~ 6

Y_one_hot = tf.one_hot(Y, nb_classes) # rank++ // 2 -> 3
print("one_hot", Y_one_hot)

# [0, 6] => [1000000, ... , 0000001)

Y_one_hot = tf.reshape(Y_one_hot, [-1, nb_classes]) # count * 7, rank-- // 3 -> 2
print("reshape", Y_one_hot)

W = tf.Variable(tf.random_normal([16, nb_classes]), name='weight')
b = tf.Variable(tf.random_normal([nb_classes]), name='bias')

# tf.nn.softmax computes softmax activations
# softmax = exp(logits) / reduce_sum(exp(logits), dim)
logits = tf.matmul(X, W) + b
hypothesis = tf.nn.softmax(logits)

# Cross entropy cost/loss
cost_i = tf.nn.softmax_cross_entropy_with_logits(logits=logits,
                                                 labels=Y_one_hot)
cost = tf.reduce_mean(cost_i)
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

prediction = tf.argmax(hypothesis, 1)
correct_prediction = tf.equal(prediction, tf.argmax(Y_one_hot, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
# Launch graph
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for step in range(2000):
        sess.run(optimizer, feed_dict={X: x_data, Y: y_data})
        if step % 100 == 0:
            loss, acc = sess.run([cost, accuracy], feed_dict={
                                 X: x_data, Y: y_data})
            print("Step: {:5}\tLoss: {:.3f}\tAcc: {:.2%}".format(
                step, loss, acc))

    # Let's see if we can predict
    pred = sess.run(prediction, feed_dict={X: x_data})
    # y_data: (N,1) = flatten => (N, ) matches pred.shape
    for p, y in zip(pred, y_data.flatten()):
        print("[{}] Prediction: {} True Y: {}".format(p == int(y), p, int(y)))

(101, 16) (101, 1)
one_hot Tensor("one_hot:0", shape=(?, 1, 7), dtype=float32)
reshape Tensor("Reshape:0", shape=(?, 7), dtype=float32)
Step:     0	Loss: 5.106	Acc: 37.62%
Step:   100	Loss: 0.800	Acc: 79.21%
Step:   200	Loss: 0.486	Acc: 88.12%
Step:   300	Loss: 0.349	Acc: 90.10%
Step:   400	Loss: 0.272	Acc: 94.06%
Step:   500	Loss: 0.222	Acc: 95.05%
Step:   600	Loss: 0.187	Acc: 97.03%
Step:   700	Loss: 0.161	Acc: 97.03%
Step:   800	Loss: 0.140	Acc: 97.03%
Step:   900	Loss: 0.124	Acc: 97.03%
Step:  1000	Loss: 0.111	Acc: 97.03%
Step:  1100	Loss: 0.101	Acc: 99.01%
Step:  1200	Loss: 0.092	Acc: 100.00%
Step:  1300	Loss: 0.084	Acc: 100.00%
Step:  1400	Loss: 0.078	Acc: 100.00%
Step:  1500	Loss: 0.072	Acc: 100.00%
Step:  1600	Loss: 0.068	Acc: 100.00%
Step:  1700	Loss: 0.064	Acc: 100.00%
Step:  1800	Loss: 0.060	Acc: 100.00%
Step:  1900	Loss: 0.057	Acc: 100.00%
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 3 True Y: 3
[True] Prediction: 0 True Y: 0
[True] Predi